In [250]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import copy
from scipy.linalg import svd
import random
from sklearn import preprocessing
import pickle
from scipy.optimize import fsolve
from scipy.special import gamma as g
from scipy.optimize import minimize
from _util import *

In [320]:
def meanvar_meanprex(mean, var):
    "mean-variance to precision"
    return 1/(mean*(1-mean)/var-1)

def logistic(x):
    return np.exp(x) / (np.exp(x) + 1)
def logit(x):
    return np.log(x/(1-x))
def likelhood_beta(psi,a,b):
    out = np.log(g(psi)/(g(b*psi)*g((1-b)*psi)))+(b*psi-1)*np.log(a)+((1-b)*psi-1)*np.log(1-a)
    return -np.sum(out)
from fancyimpute import MatrixFactorization

#########################
def v_2_y(mat):
    temp = 2 / (1+mat) - 1
    return logit(temp)

def v_2_theta(v):
    return 1/(1+v)
def y_2_theta(y):
    return (logistic(y)+1)/2
def theta_2_v(theta):
    return 1/theta - 1

In [312]:
def Beta_loglike(y_mean, v_obs, phi_beta):
    alphas, betas = beta_reparameterize(y_2_theta(y_mean),phi_beta)
    from scipy.stats import beta
    theta_obs = v_2_theta(v_obs)
    return sum([np.log(beta.pdf(y, a, b)) for y, a, b in zip(theta_obs, alphas, betas)])

In [252]:
data = pd.read_csv('MNL_real_dataset/ratings.dat', sep='::', names = ["user_id","movie_id","rating","timestamp"]  , encoding='latin-1')

## Goal 
* Approach I: get feature and utility from real data, but do not assume relationship between them (though we need to be careful when get featurrs)
    1. apply low-rank matrix completion and SVD to extract the feature of each movie as $x_i$, from the training dataset. 
    2. estimate the utility of each movie as $v_i$, from the testing dataset
    3. simulate customer's choice with an MNL model parameterized by $\{v_i\}$. Some algorithms might additionally use $\{x_i\}$ if they believe it helps.
* Approach II: get feature and theta from real data, and simulate following our model

1. Impute only training, get features from the continuous rating + [d, X_transform, with_intercept]
2. Impute All, then binarize, get features from traiing, and predict the testing + d = 5, [X_transform, with_intercept]
    1. try d = 5 to impute but d = 3 as feature

1. What we need: a good but not perfect relationship between x and v.

## Our model
* $\theta_i \sim Beta(\frac{logistic(x_i^T \gamma)+ 1}{2}, \psi)$
* $E(\theta_i|x_i) = \frac{logistic(x_i^T \gamma)+ 1}{2}$
* $\theta_i = (1+v_i)^{-1}$
* $logit(2E(\theta_i|x_i)-1) = x_i^T \gamma$
* $logit(p) = log(p/(1-p))$
* $logit(2E((1+v_i)^{-1}|x_i)-1) = x_i^T \gamma$
* though not rigrous, we can probably follow this to transform the entry

## SVD
* When we use SVD to extract the features $x_i$ and then use mean ratings as $v_i$, we assume $E(v_i|x_i) = x_i^T \gamma$, which is not fair to us (when we design feature in real datasets, we also would like to make sure our model is close to being correct)
    * but we can still have a reasonable model even with this? r2 says so. 
    * If we go with approach II, regression here is just a check

## Else
* v \in [0,1]; various way to normalize and define v
* larger variations (but feature should be accurate)
* Intercept

In [335]:
L=1000
selected_movies = list(data.movie_id.value_counts().index[:L])
selected_users = list(data[data.movie_id.isin(selected_movies)].user_id.value_counts().index[:])
selected_data=data[data.movie_id.isin(selected_movies)][data.user_id.isin(selected_users)]
len(selected_data)
selected_movies = [a-1 for a in selected_movies]
selected_users = [a-1 for a in selected_users]

In [336]:
num_users = 6040
num_movies = 3952
#W_feedback_matrix = np.zeros((num_users, num_movies))
W_feedback_ratings = np.empty((num_users, num_movies))
W_feedback_ratings[:] = np.nan
W_feedback_ratings[np.array(data.user_id-1), np.array(data.movie_id-1)] = np.array(data.rating/5)
# W_feedback_ratings[np.array(data.user_id-1), np.array(data.movie_id-1)] = np.array(data.rating > 3).astype(int)
# randomly permuting the users
random.shuffle(selected_users)
W_feedback_ratings = W_feedback_ratings[selected_users,:]
W_feedback_ratings = W_feedback_ratings[:,selected_movies]


In [337]:
# can be updated, if needed
# something wrong here!!!
W_train = W_feedback_ratings[:num_movies//2,:]
W_test = W_feedback_ratings[num_movies//2:,:]

In [419]:
def fit_best_gamma_phi(p, X, v):
    """ suppose we can see all v w/o error and in hindsight, find the best phi, gamma; for debug purpose"""
    import pymc3 as pm
    n_init = 2000
    n_tune = 200
    chains = 1
    n_sample = 2000

    with pm.Model() as Normal_Geom:
        gamma_temp = pm.MvNormal('gamma', mu=np.zeros(p), cov=np.identity(p),shape=p)
        phi = pm.Beta('phi', alpha= 1, beta=1, shape=1)
        alpha_temp = pm.math.dot(X, gamma_temp)
        mean_theta = (logistic(alpha_temp)+1)/2
        alpha_Beta, beta_Beta = beta_reparameterize(mean_theta, phi)
        theta = v_2_theta(v)
        theta = pm.Beta('theta', alpha= alpha_Beta, beta=beta_Beta, shape=L, observed = theta)
        trace = pm.sample(n_sample, tune = n_tune, chains = chains
                          , cores = 1, progressbar = 1, init='adapt_diag',
                          target_accept=0.95, trace = None);
    return {'gamma' : np.mean(trace["gamma"], 0), 'phi' : np.mean(trace["phi"], 0)}

In [263]:
W_feedback_ratings.shape

(6040, 1000)

In [262]:
num_movies

3952

### Extract Features (from Training)

In [ ]:
## Impute
for d in [5, 10]:
     W_train_softimpute = MatrixFactorization(rank=d,
        learning_rate=0.05,
             max_iters=50,
             shrinkage_value=0,
             min_value=None,
             max_value=None,
             verbose=True).fit_transform(W_feedback_ratings)
    dump(W_train_softimpute, 'MNL_real_dataset/W_train_softimpute_back_{}'.format(d))

In [338]:
d = 5

In [339]:
W_train_softimpute = load('MNL_real_dataset/W_train_softimpute_back_{}'.format(d))

### Check the regression's quality (on Testing)

In [353]:
## MNL utility estimates
W_train_mean = np.nanmean(W_train, axis = 0) 
W_test_mean = np.nanmean(W_test, axis = 0) 
W_train_binary_mean = np.nanmean(W_train >= 0.8, axis = 0) 
W_test_binary_mean = np.nanmean(W_test >= 0.8, axis = 0) 

In [424]:
v = W_test_mean
y = v_2_y(v)

In [ ]:
for X_transform in ['l2']:
    for with_intercept in [1]:
        for d in [5]:
            W_train_softimpute = load('MNL_real_dataset/W_train_softimpute_back_{}'.format(d))
            U, s, VT = svd(W_train_softimpute)
            X = movie_features = np.matmul(VT.T[:,:d],np.diag(s[:d]))
            X = preprocessing.normalize(X, norm='l2') 

            if with_intercept:
                X = sm.add_constant(X)
            results = fit_best_gamma_phi(X.shape[1], X, v)
            gamma = results['gamma']
            best_phi = results['phi'][0]
            y_mean = X.dot(gamma)
            v_y_mean = theta_2_v(y_2_theta(y_mean))
            v_y = theta_2_v(y_2_theta(y))

            print("fitting R2 = {:.2f}".format(r2_score(y, y_mean)))
            print("fitting R2 for v = {:.2f}".format(r2_score(v_y_mean, v_y)))

            n_train = 750
            results2 = fit_best_gamma_phi(X.shape[1], X[:n_train], v[:n_train]) 
            y1 = y[n_train:]
            y2 = X[n_train:].dot(results2['gamma'])
            print("prediction R2 = {:.2f}".format(r2_score(y1, y2)))
            v1 = theta_2_v(y_2_theta(y1))
            v2 = theta_2_v(y_2_theta(y2))
            print("prediction R2 for v = {:.2f}".format(r2_score(v1, v2)))

            print('max_para = {:.2f}'.format(max(abs(gamma))))
            print('best_phi = {:.2f}'.format(best_phi))

            out = {
                   'W_test_mean': W_test_mean,
                   'movie_features':X,
                   'true_gamma_wrt_test': gamma,
                   'true_phi_wrt_test': best_phi,
            }

            fp = 'MNL_real_dataset/MNL_realdata_d_{}_X_transform_{}_with_intercept_{}'.format(d, X_transform, with_intercept)
            dump(out, fp)    
            print("\n")
            # _binary
        print("*" * 100)